In [1]:
import argparse
import time
import datetime
import torch
import torch_ac
import tensorboardX
import sys
sys.path.insert(0,'/root/server/Semantic-Loss/complex_constraints/curriculum learning/automatic-curriculum/')
import utils
import auto_curri as ac
from model import ACModel
import numpy as np

In [2]:
class parser:
    def __init__(self):
        self.env='MiniGrid-BlockedUnlockPickup-v0'#'MiniGrid-Unlock-v0'
        self.curriculum=None#'BlockedUnlockPickup'
        self.model=None
        self.seed=1
        self.log_interval=1
        self.save_interval=10
        self.procs=16
        self.frames=10**7
        self.epochs=4
        self.batch_size=256
        self.frames_per_proc=128
        self.discount=0.99
        self.lr=0.001
        self.gae_lambda=0.95
        self.entropy_coef=0.01
        self.value_loss_coef=0.5
        self.max_grad_norm=0.5
        self.adam_eps=1e-8
        self.clip_eps=0.2
        self.lpe="Linreg"
        self.lpe_alpha=0.1
        self.lpe_K=10
        self.acp="MR"
        self.acp_MR_K=10
        self.acp_MR_power=6
        self.acp_MR_pot_prop=0.5
        self.acp_MR_att_pred=0.2
        self.acp_MR_att_succ=0.05
        self.a2d="Prop"
        self.a2d_eps=0.1
        self.a2d_tau=4e-4
args = parser()

# Save the arguments in a table

config_hash = utils.save_config_in_table(args, "config_rl")

# Set run dir

name = args.env or args.curriculum
date = datetime.datetime.now().strftime("%y-%m-%d-%H-%M-%S")
default_model_name = f"{name}_seed{args.seed}_{config_hash}_{date}"

model_name = args.model or default_model_name
model_dir = utils.get_model_dir(model_name)

# Load loggers and Tensorboard writer

txt_logger = utils.get_txt_logger(model_dir)
csv_file, csv_logger = utils.get_csv_logger(model_dir)
tb_writer = tensorboardX.SummaryWriter(model_dir)

# Log command and all script arguments

txt_logger.info("{}\n".format(" ".join(sys.argv)))
txt_logger.info("{}\n".format(args))
txt_logger.info("Config hash: {}\n".format(config_hash))

# Set seed for all randomness sources

utils.seed(args.seed)

# Set device

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
txt_logger.info(f"Device: {device}\n")

# Load environments

if args.env is not None:
    envs = []
    for i in range(args.procs):
        envs.append(utils.make_env(args.env, args.seed + 10000 * i))

elif args.curriculum is not None:
    # Load curriculum
    G, env_ids, init_min_returns, init_max_returns = utils.get_curriculum(args.curriculum)

    # Make distribution computer
    compute_dist = ac.make_dist_computer(
                        len(env_ids), args.lpe, args.lpe_alpha, args.lpe_K,
                        args.acp, G, init_min_returns, init_max_returns, args.acp_MR_K, args.acp_MR_power,
                        args.acp_MR_pot_prop, args.acp_MR_att_pred, args.acp_MR_att_succ,
                        args.a2d, args.a2d_eps, args.a2d_tau)

    # Make polymorph environments
    penv_head = ac.PolyEnvHead(args.procs, len(env_ids), compute_dist)
    envs = []
    for i in range(args.procs):
        seed = args.seed + 10000 * i
        envs.append(ac.PolyEnv(utils.make_envs_from_curriculum(env_ids, seed), penv_head.remotes[i], seed))

txt_logger.info("Environments loaded\n")

# Load training status

try:
    status = utils.get_status(model_dir)
except OSError:
    status = {"num_frames": 0, "update": 0}
txt_logger.info("Training status loaded\n")

# Load observations preprocessor
obs_space, preprocess_obss = utils.get_obss_preprocessor(envs[0].observation_space)
txt_logger.info("Observations preprocessor loaded")

# Load model

acmodel = ACModel(obs_space, envs[0].action_space)
if "model_state" in status:
    acmodel.load_state_dict(status["model_state"])
acmodel.to(device)
txt_logger.info("Model loaded\n")
txt_logger.info("{}\n".format(acmodel))

# Load algo

algo = torch_ac.PPOAlgo(envs, acmodel, device, args.frames_per_proc, args.discount, args.lr, args.gae_lambda,
                        args.entropy_coef, args.value_loss_coef, args.max_grad_norm, 1,
                        args.adam_eps, args.clip_eps, args.epochs, args.batch_size, preprocess_obss,
                        utils.reshape_reward)
if "optimizer_state" in status:
    algo.optimizer.load_state_dict(status["optimizer_state"])
txt_logger.info("Optimizer loaded\n")

# Train model

num_frames = status["num_frames"]
update = status["update"]
start_time = time.time()
rewards=[]
while num_frames < args.frames:
    # Update model parameters

    update_start_time = time.time()
    exps, logs1 = algo.collect_experiences()
    logs2 = algo.update_parameters(exps)
    logs = {**logs1, **logs2}
    if args.curriculum is not None:
        penv_head.update_dist()
        print(penv_head.dist)
    update_end_time = time.time()

    num_frames += logs["num_frames"]
    update += 1
    
    # Print logs

    if update % args.log_interval == 0:
        fps = logs["num_frames"] / (update_end_time - update_start_time)
        duration = int(time.time() - start_time)
        txt_logger.info("U {} | F {} | FPS {:04.0f} | D {}|reward {}".format(update, num_frames, fps, duration,np.array(logs['return_per_episode']).mean()))
        rewards.append(np.array(logs['return_per_episode']).mean())
        header = []
        data = []

        if args.env is not None:
            header += ["perf"]
            data += [utils.synthesize(logs["return_per_episode"])["mean"]]
        elif args.curriculum is not None:
            for i, env_id in enumerate(env_ids):
                header += ["proba/{}".format(env_id)]
                data += [penv_head.dist[i]]
                header += ["perf/{}".format(env_id)]
                data += [None]
                if i in penv_head.synthesized_returns.keys():
                    data[-1] = penv_head.synthesized_returns[i]
                if args.acp in ["LP", "MR"]:
                    header += ["lp/{}".format(env_id)]
                    data += [compute_dist.compute_att.lps[i]]
                    header += ["attention/{}".format(env_id)]
                    data += [compute_dist.compute_att.atts[i]]
                if args.acp in ["MR"]:
                    header += ["max_perf/{}".format(env_id)]
                    data += [compute_dist.compute_att.max_perfs[i]]
                    header += ["na_lp/{}".format(env_id)]
                    data += [compute_dist.compute_att.na_lps[i]]
                    header += ["mr/{}".format(env_id)]
                    data += [compute_dist.compute_att.mrs[i]]
                    header += ["anc_mr/{}".format(env_id)]
                    data += [compute_dist.compute_att.anc_mrs[i]]
                    header += ["learning_state/{}".format(env_id)]
                    data += [compute_dist.compute_att.learning_states[i]]
                    header += ["pre_attention/{}".format(env_id)]
                    data += [compute_dist.compute_att.pre_atts[i]]

        if status["num_frames"] == 0:
            csv_logger.writerow(header)
        csv_logger.writerow(data)
        csv_file.flush()

        for field, value in zip(header, data):
            if value is not None:
                tb_writer.add_scalar(field, value, num_frames)

    # Save status

    if args.save_interval > 0 and update % args.save_interval == 0:
        status = {"num_frames": num_frames, "update": update,
                  "model_state": acmodel.state_dict(), "optimizer_state": algo.optimizer.state_dict()}
        utils.save_status(status, model_dir)
        txt_logger.info("Status saved")
#https://github.com/lcswillems/automatic-curriculum

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-db5e0ddd-012e-4123-abef-d5f72e7ec20c.json


Config hash: 807b93d9cf

Device: cuda

Environments loaded

Training status loaded

Observations preprocessor loaded
Model loaded

ACModel(
  (image_conv): Sequential(
    (0): Conv2d(3, 16, kernel_size=(2, 2), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1))
    (4): ReLU()
    (5): Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1))
    (6): ReLU()
  )
  (actor): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=7, bias=True)
  )
  (critic): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
)



OSError: [Errno 12] Cannot allocate memory